In [1]:
import numpy as np
import pandas as pd
import openpyxl
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency
from geopy.geocoders import Nominatim


In [47]:
raw = pd.read_excel('raw_data.xlsx')
raw

,Timestamp,Age,sex,Nationality,base,Mother_tongue,Other_languages,education,Part_full,working_student,Academic_major,Bootcamp_subject,Institute,Graduation_time,local_language,Familirity_rate,Intention,Employee,Time_employee,Apply_employee,Work_exprience_employee,Time_hunter,Apply_hunter,Work_exprience_hunter,Upskilled_employee_try,Upskilled_employee_succeed,Work_exprience_upskilled,Time_upskilled,Apply_upskilled,Work_exprience_upskilled_hunter,Time_upskilled_hunter,Apply_upskilled_hunter,Subjective_technical,Subjective_soft,Subjective_language,Subjective_academia,Subjective_bootcamp,CV,Email
0,2023/11/03 10:31:24 AM GMT+1,Between 36 and 45,Male,British,Netherlands,English,Dutch,Bachelor's degree,Full time,No,Fine Art,UI UX,Ironhack,2024-02-03 00:00:00,1,Almost nothing,Landing a job in a related field,Not yet,NaN,NaN,NaN,Not started yet.,NaN,less than 1 year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4,2,3,2,https://drive.google.com/u/0/open?usp=forms_we...,NaN
1,2023/11/03 10:44:42 AM GMT+1,Between 24 and 35,Male,German,Portugal,German,"english, dutch",Bachelor's degree,Full time,Yes - Part time,Communication,WebDev,Lisbon,2023-12-05 00:00:00,0,Almost nothing,Landing a job in a related field,Not yet,NaN,NaN,NaN,Not started yet.,Less than 50,between 1 year and 3 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,4,4,3,2,https://drive.google.com/u/0/open?usp=forms_we...,NaN
2,2023/11/03 11:11:17 AM GMT+1,Between 24 and 35,Female,Iranian,Germany,Persian,English,Bachelor's degree,Full time,No,Civil engineering,web development,Spiced Academy,2023-08-09 00:00:00,0,Between 20% and 50%,Landing a job in a related field,Not yet,NaN,NaN,NaN,Less than 6 months,Less than 50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,2,1,1,NaN,NaN
3,2023/11/03 11:52:15 AM GMT+1,Between 24 and 35,Female,Swiss,Germany,German,"English, Swiss-German",Some college or associate's degree,Full time,No,Early Childhood Education,UX/UI Design,Ironhack,2023-08-11 00:00:00,1,Less than 20%,I have a job in a related field and just inten...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,No,NaN,NaN,NaN,Less than 1 year,Less than 6 months,Between 50 and 100,4,3,2,1,4,NaN,NaN
4,2023/11/03 2:33:26 PM GMT+1,Between 46 and 60,Male,German,Germany,German,"English, Spanish",Bachelor's degree,Full time,No,Science of Art,Fullstack Development,Spiced Academy,2023-07-06 00:00:00,1,Almost nothing,Landing a job in a related field,Not yet,NaN,NaN,NaN,Less than 6 months,Less than 50,less than 1 year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,4,4,3,3,NaN,thoralf.barth@gmail.com
5,2023/11/03 5:25:32 PM GMT+1,Between 36 and 45,Male,Ethiopian,Germany,Amharic,English,Bachelor's degree,Full time,No,Computer science,Cop up to modern programming skills,Spiced academy,2023-07-06 00:00:00,0,More than 50%,Landing a job in a related field,Not yet,NaN,NaN,NaN,Less than 6 months,Between 50 and 100,more than 3 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2,3,3,2,NaN,NaN
6,2023/11/06 7:26:45 AM GMT+1,Between 24 and 35,Female,Iranian,Germany,Persian,English,Bachelor's degree,Full time,No,Conservation and restoration of cultural heritage,Web developement,Ironhack,2022-06-03 00:00:00,0,Almost nothing,Landing a job in a related field,Yes,1 week,Less than 50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,5,3,3,4,https://drive.google.com/u/0/open?usp=forms_we...,NaN
7,2023/11/06 10:22:03 AM GMT+1,Between 24 and 35,Female,Iranian,Germany,Azerbaijani,Persian. English. Turkish,Master's degree,Full time,Yes - Part time,MBA in project management,UI/UX,Ironhack,2021-12-10 00:00:00,0,Less than 20%,Landing a job in a related field,Yes,less than 6 months,Less than 50,between 1year and 3 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4,2,1,3,https://drive.google.com/u/0/open?usp=forms_we...,NaN
8,2023/11/06 10:31:03 AM GMT+1,Between 24 and 35,Female,Romanian,Germany,Romanian,"English, Spanish",Master's degree,Part time,Yes - Part time,Natural Sciences,Data Analytics,Ironhack,2022-07-31 00:00:00,0,Less than 20%,Landing a job in a related field,Yes,less than 6 month

In [48]:
raw.columns

Index(['Timestamp', 'Age', 'sex', 'Nationality', 'base', 'Mother_tongue',
       'Other_languages', 'education', 'Part_full', 'working_student',
       'Academic_major', 'Bootcamp_subject', 'Institute', 'Graduation_time',
       'local_language', 'Familirity_rate', 'Intention', 'Employee',
       'Time_employee', 'Apply_employee', 'Work_exprience_employee',
       'Time_hunter', 'Apply_hunter', 'Work_exprience_hunter',
       'Upskilled_employee_try', 'Upskilled_employee_succeed',
       'Work_exprience_upskilled', 'Time_upskilled', 'Apply_upskilled',
       'Work_exprience_upskilled_hunter', 'Time_upskilled_hunter',
       'Apply_upskilled_hunter', 'Subjective_technical', 'Subjective_soft',
       'Subjective_language', 'Subjective_academia', 'Subjective_bootcamp',
       'CV', 'Email'],
      dtype='object')

In [49]:
raw['Intention'].replace('I have a job in a related field and just intended to upskill myself','Landing a job in a related field',inplace=True)

raw['Work_exprience_upskilled'] = raw['Work_exprience_employee']
raw['Time_upskilled'] = raw['Time_employee']
raw['Apply_upskilled'] = raw['Apply_employee']

raw['Work_exprience_upskilled_hunter' ] = raw['Work_exprience_hunter']
raw['Time_upskilled_hunter'] = raw['Time_hunter']
raw['Apply_upskilled_hunter'] = raw['Apply_hunter']




In [50]:
raw.drop(columns=['Timestamp','Familirity_rate','Upskilled_employee_try', 'Upskilled_employee_succeed',
       'Work_exprience_upskilled', 'Time_upskilled', 'Apply_upskilled',
       'Work_exprience_upskilled_hunter', 'Time_upskilled_hunter',
       'Apply_upskilled_hunter','CV', 'Email','Intention'],inplace=True)

In [51]:
geolocator = Nominatim(user_agent="myGeocoder")
latitude = []
longitude = []

for country in raw['base']:
    location = geolocator.geocode(country)
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    
raw['latitude'] = latitude
raw['longitude'] = longitude

In [52]:
raw.columns

Index(['Age', 'sex', 'Nationality', 'base', 'Mother_tongue', 'Other_languages',
       'education', 'Part_full', 'working_student', 'Academic_major',
       'Bootcamp_subject', 'Institute', 'Graduation_time', 'local_language',
       'Employee', 'Time_employee', 'Apply_employee',
       'Work_exprience_employee', 'Time_hunter', 'Apply_hunter',
       'Work_exprience_hunter', 'Subjective_technical', 'Subjective_soft',
       'Subjective_language', 'Subjective_academia', 'Subjective_bootcamp',
       'latitude', 'longitude'],
      dtype='object')

In [53]:
raw['local_language'] = [True if i == 1 else False for i in raw['local_language'] ]

In [54]:
raw.shape

(31, 28)

In [55]:
pd.set_option('display.max_columns', None)
raw

,Age,sex,Nationality,base,Mother_tongue,Other_languages,education,Part_full,working_student,Academic_major,Bootcamp_subject,Institute,Graduation_time,local_language,Employee,Time_employee,Apply_employee,Work_exprience_employee,Time_hunter,Apply_hunter,Work_exprience_hunter,Subjective_technical,Subjective_soft,Subjective_language,Subjective_academia,Subjective_bootcamp,latitude,longitude
0,Between 36 and 45,Male,British,Netherlands,English,Dutch,Bachelor's degree,Full time,No,Fine Art,UI UX,Ironhack,2024-02-03 00:00:00,True,Not yet,NaN,NaN,NaN,Not started yet.,NaN,less than 1 year,4,4,2,3,2,52.243498,5.634323
1,Between 24 and 35,Male,German,Portugal,German,"english, dutch",Bachelor's degree,Full time,Yes - Part time,Communication,WebDev,Lisbon,2023-12-05 00:00:00,False,Not yet,NaN,NaN,NaN,Not started yet.,Less than 50,between 1 year and 3 years,5,4,4,3,2,39.662165,-8.135352
2,Between 24 and 35,Female,Iranian,Germany,Persian,English,Bachelor's degree,Full time,No,Civil engineering,web development,Spiced Academy,2023-08-09 00:00:00,False,Not yet,NaN,NaN,NaN,Less than 6 months,Less than 50,NaN,2,3,2,1,1,40.420348,-79.116698
3,Between 24 and 35,Female,Swiss,Germany,German,"English, Swiss-German",Some college or associate's degree,Full time,No,Early Childhood Education,UX/UI Design,Ironhack,2023-08-11 00:00:00,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,3,2,1,4,40.420348,-79.116698
4,Between 46 and 60,Male,German,Germany,German,"English, Spanish",Bachelor's degree,Full time,No,Science of Art,Fullstack Development,Spiced Academy,2023-07-06 00:00:00,True,Not yet,NaN,NaN,NaN,Less than 6 months,Less than 50,less than 1 year,5,4,4,3,3,40.420348,-79.116698
5,Between 36 and 45,Male,Ethiopian,Germany,Amharic,English,Bachelor's degree,Full time,No,Computer science,Cop up to modern programming skills,Spiced academy,2023-07-06 00:00:00,False,Not yet,NaN,NaN,NaN,Less than 6 months,Between 50 and 100,more than 3 years,4,2,3,3,2,40.420348,-79.116698
6,Between 24 and 35,Female,Iranian,Germany,Persian,English,Bachelor's degree,Full time,No,Conservation and restoration of cultural heritage,Web developement,Ironhack,2022-06-03 00:00:00,False,Yes,1 week,Less than 50,NaN,NaN,NaN,NaN,3,5,3,3,4,40.420348,-79.116698
7,Between 24 and 35,Female,Iranian,Germany,Azerbaijani,Persian. English. Turkish,Master's degree,Full time,Yes - Part time,MBA in project management,UI/UX,Ironhack,2021-12-10 00:00:00,False,Yes,less than 6 months,Less than 50,between 1year and 3 years,NaN,NaN,NaN,4,4,2,1,3,40.420348,-79.116698
8,Between 24 and 35,Female,Romanian,Germany,Romanian,"English, Spanish",Master's degree,Part time,Yes - Part time,Natural Sciences,Data Analytics,Ironhack,2022-07-31 00:00:00,False,Yes,less than 6 months,Less than 50,between 1year and 3 years,NaN,NaN,NaN,5,3,2,4,1,40.420348,-79.116698
9,Between 24 and 35,Female,American,The United States,English,none,Bachelor's degree,Full time,No,Sociology and Anthropology,UX/UI,Ironhack,2023-12-08 00:00:00,True,Not yet,NaN,NaN,NaN,Not started yet.,Less than 50,NaN,4,4,2,2,2,47.828163,-122.598365


In [35]:
raw.to_excel('raw_EDA.xlsx')

In [70]:

contingency_table = pd.crosstab(raw['education'],raw['Employee'])

print("Contingency Table:")
print(contingency_table)

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("\nChi-squared statistic:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies table:")
print(pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns))

# Interpret the results
alpha = 0.05
if p < alpha:
    print("\nThere is a significant association between the two categorical variables (Gender and Preference).")
else:
    print("\nThe two categorical variables are independent.")

Contingency Table:
Employee                            Not yet  Yes
education                                       
Bachelor's degree                         7    4
Master's degree                           2    3
Some college or associate's degree        2    0

Chi-squared statistic: 2.239905548996458
p-value: 0.32629520371390647
Degrees of freedom: 2
Expected frequencies table:
Employee                             Not yet       Yes
education                                             
Bachelor's degree                   6.722222  4.277778
Master's degree                     3.055556  1.944444
Some college or associate's degree  1.222222  0.777778

The two categorical variables are independent.


In [77]:
contingency_table = pd.crosstab(raw['local_language'],raw['Employee'])

print("Contingency Table:")
print(contingency_table)

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("\nChi-squared statistic:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies table:")
print(pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns))

# Interpret the results
alpha = 0.05
if p < alpha:
    print("\nThere is a significant association between the two categorical variables (Gender and Preference).")
else:
    print("\nThe two categorical variables are independent.")

Contingency Table:
Employee        Not yet  Yes
local_language              
False                 5    3
True                  6    4

Chi-squared statistic: 0.0
p-value: 1.0
Degrees of freedom: 1
Expected frequencies table:
Employee         Not yet       Yes
local_language                    
False           4.888889  3.111111
True            6.111111  3.888889

The two categorical variables are independent.
